# Clustering

![ml workflow](https://docs.google.com/drawings/d/e/2PACX-1vQ1XLwesZbm_TuDBPFRvbHa4XcjucvtExy3LXE05WnaAw-s6BDVQnnd4lAEUW1Qy6bs6FythuJdFVqP/pub?w=1165&h=662)

Let's now try out some **Clustering**. A clusters or group is a collection of data points into which our data points are organized. This is opposed to predicting a continuous value (like weight, temperature, or stock price).
Classification attempts to predict whether a certain sample belongs to one class or another (e.g., fraud or not fraud). 

For our example, we will be trying to cluster the data of fleet information fleet_data.csv. We have a Driver Id and the mean distance driven per driver. Speeding_Feature is a mean percentage of time during which the driver is driving 5+ miles per hour faster than the speed limit.

## Imports

In [ ]:
import "fmt"
import "os"
import "strings"
import "strconv"
import "encoding/csv"
import "image/color"
import "image"
import "image/png"
import "io/ioutil"
import "gonum.org/v1/gonum/floats"
import "gonum.org/v1/plot"
import "gonum.org/v1/plot/plotter"
import "gonum.org/v1/plot/plotutil"
import "gonum.org/v1/plot/vg"
import "github.com/kniren/gota/dataframe"
import "github.com/mash/gokmeans"

## Convenience Functions

In [ ]:
// GetGraph returns the bytes corresponding to a
// saved plot.
func GetGraph(graphName string) ([]byte, error) {
    
    // Open the file.
    infile, err := os.Open(graphName)
    if err != nil {
        return nil, err
    }
    
    // Read in the contents of the file.
    bytes, err := ioutil.ReadAll(infile)
    if err != nil {
        return nil, err
    }
    
    // Close the file.
    infile.Close()
    
    return bytes, err
}

## Import the Data

In [ ]:
file, err := os.Open("../data/fleet_data.csv")
if err != nil {
    fmt.Println(err)
}

In [ ]:
dataset := dataframe.ReadCSV(file)

In [ ]:
file.Close()

In [ ]:
fmt.Println(dataset)

## Profile Data

In [ ]:
summary := dataset.Describe()

In [ ]:
fmt.Println(summary)

In [ ]:
// Create a histogram for each of the columns in the dataset.
for _, colName := range dataset.Names() {
    if colName != "Driver_ID" {
        // Create a plotter.Values value and fill it with the
        // values from the respective column of the dataframe.
        plotVals := make(plotter.Values, dataset.Nrow())
        for i, floatVal := range dataset.Col(colName).Float() {
            plotVals[i] = floatVal
        }

        // Make a plot and set its title.
        p, err := plot.New()
        if err != nil {
            fmt.Println("error creating plot",err)
        }
        p.Title.Text = fmt.Sprintf("Histogram of %s", colName)

        // Create a histogram of our values.
        h, err := plotter.NewHist(plotVals, 16)
        if err != nil {
            fmt.Println("error creating histogram",err)
        }

        // Normalize the histogram.
        h.Normalize(1)

        // Add the histogram to the plot.
        p.Add(h)

        // Save the plot to a PNG file.
        if err := p.Save(4*vg.Inch, 4*vg.Inch, colName+"_hist.png"); err != nil {
            fmt.Println("error saving",err)
        }
    }
}

In [ ]:
graphCal, err :=GetGraph("Distance_Feature_hist.png")
if err != nil {
    fmt.Println(err)
}
display.PNG(graphCal)

In [ ]:
graphCar, err :=GetGraph("Speeding_Feature_hist.png")
if err != nil {
    fmt.Println(err)
}
display.PNG(graphCar)

In [ ]:
// Function to create an scatter plot from 2 column names 
func graphScatterFromColumns(columnA,columnB string){
   yVals := dataset.Col(columnA).Float()
   // pts will hold the values for plotting
   pts := make(plotter.XYs, dataset.Nrow())
   // Fill pts with data.
   for i, floatVal := range dataset.Col(columnB).Float() {
       pts[i].X = floatVal
       pts[i].Y = yVals[i]
   }
   // Create the plot.
   p, err := plot.New()
   if err != nil {
       fmt.Println(err)
   }
   p.X.Label.Text = columnB
   p.Y.Label.Text = columnA
   p.Add(plotter.NewGrid())
   s, err := plotter.NewScatter(pts)
   if err != nil {
       fmt.Println(err)
   }
   s.GlyphStyle.Color = color.RGBA{R: 255, B: 128, A: 255}
   s.GlyphStyle.Radius = vg.Points(3)
   // Save the plot to a PNG file.
   p.Add(s)
   if err := p.Save(4*vg.Inch, 4*vg.Inch, "scatter_"+columnA+"_vs_"+columnB+".png"); err !=
   nil {
       fmt.Println(err)
   }
}

In [ ]:
for _, colNameA := range dataset.Names() {
    for _, colNameB := range dataset.Names() {
        if colNameA != "Driver_ID" && colNameB != "Driver_ID" && colNameA != colNameB  {
            graphScatterFromColumns(colNameA,colNameB)
        }
    }
}

In [ ]:
graphCalVCar, err :=GetGraph("scatter_Distance_Feature_vs_Speeding_Feature.png")
if err != nil {
    fmt.Println(err)
}
display.PNG(graphCalVCar)

## KMEANS

In [ ]:
// Initialize a slice of gokmeans.Node's to
// hold our input data.
var data []gokmeans.Node
// Loop over the records creating our slice of
// gokmeans.Node's.
for _,record := range dataset.Select([]int{1,2}).Records(){
    // Read in our record and check for errors.
    if record[0] != "Distance_Feature"{
        // Initialize a point.
        var point []float64
        // Fill in our point.
        for i := 0; i < 2; i++ {
            // Parse the float value.
           val, err := strconv.ParseFloat(record[i], 64)
           if err != nil {
                fmt.Println(err)
            }
            // Append this value to our point.
            point = append(point, val)
        }
        // Append our point to the data.
        data = append(data, gokmeans.Node{point[0], point[1]})
    }
}
// Generate our clusters with k-means.
success, centroids := gokmeans.Train(data, 2, 50)
if !success {
   fmt.Println("Could not generate clusters")
}
// Output the centroids to stdout.
fmt.Println("The centroids for our clusters are:")
for _, centroid := range centroids {
    fmt.Println(centroid)
}

### Evaluating the generated clusters

In [ ]:
// Extract the distance column.
yVals := dataset.Col("Distance_Feature").Float()
// clusterOne and clusterTwo will hold the values for plotting.
var clusterOne [][]float64
var clusterTwo [][]float64
// Fill the clusters with data.
for i, xVal := range dataset.Col("Speeding_Feature").Float() {
    distanceOne := floats.Distance([]float64{yVals[i], xVal},centroids[0], 2)
    distanceTwo := floats.Distance([]float64{yVals[i], xVal},centroids[1], 2)
    if distanceOne < distanceTwo {
        clusterOne = append(clusterOne, []float64{xVal, yVals[i]})
    }else{
        clusterTwo = append(clusterTwo, []float64{xVal, yVals[i]})
    }
}
// pts* will hold the values for plotting
ptsOne := make(plotter.XYs, len(clusterOne))
ptsTwo := make(plotter.XYs, len(clusterTwo))
// Fill pts with data.
for i, point := range clusterOne {
    ptsOne[i].X = point[0]
    ptsOne[i].Y = point[1]
}
for i, point := range clusterTwo {
    ptsTwo[i].X = point[0]
    ptsTwo[i].Y = point[1]
}
// Create the plot.
p, err := plot.New()
if err != nil {
    fmt.Println(err)
}
p.X.Label.Text = "Speeding"
p.Y.Label.Text = "Distance"
p.Add(plotter.NewGrid())
sOne, err := plotter.NewScatter(ptsOne)
if err != nil {
    fmt.Println(err)
}
sOne.GlyphStyle.Radius = vg.Points(3)
//sOne.GlyphStyle.Shape = draw.RingGlyph{}
sTwo, err := plotter.NewScatter(ptsTwo)
if err != nil {
    fmt.Println(err)
}
sTwo.GlyphStyle.Radius = vg.Points(30)
// Save the plot to a PNG file.
p.Add(sOne, sTwo)
if err := p.Save(4*vg.Inch, 4*vg.Inch, "fleet_data_clusters.png"); err !=nil {
    fmt.Println(err)
}


In [ ]:
graphCalVCar, err :=GetGraph("fleet_data_clusters.png")
if err != nil {
    fmt.Println(err)
}
display.PNG(graphCalVCar)

### one more qunatitative evaluation

In [ ]:
// withinClusterMean calculates the mean distance between
// points in a cluster and the centroid of the cluster.
func withinClusterMean(cluster [][]float64, centroid []float64) float64 {
    // meanDistance will hold our result.
    var meanDistance float64
    // Loop over the points in the cluster.
    for _, point := range cluster {
         meanDistance += floats.Distance(point, centroid, 2) /float64(len(cluster))
    }
    return meanDistance
}


In [ ]:
// Output our within cluster metrics.
fmt.Printf("\nCluster 1 Metric: %0.2f\n", withinClusterMean(clusterOne,centroids[0]))
fmt.Printf("\nCluster 2 Metric: %0.2f\n", withinClusterMean(clusterTwo,centroids[1]))

# k-means on digits

Now let's try k-means algorithm in a more complex example. 
As it is a clustering technique, we are going to take a look at applying to a simple digits database.
We will start by loading the digits and then finding the KMeans clusters. The digits consist of 1,797 
samples with 64 features, where each of the 64 features is the brightness of one pixel in an 8×8 image.
Digits can be found at http://archive.ics.uci.edu/ml/datasets/Pen-Based+Recognition+of+Handwritten+Digits

### Data visualization

unzip the file optImg.zip in the data folder. Let's see some of the images to have one idea of what are we going to process

In [ ]:
img, err :=GetGraph("../data/optImg/optdigit_1-0.png")
if err != nil {
    fmt.Println(err)
}
display.PNG(img)

In [ ]:
img, err :=GetGraph("../data/optImg/optdigit_2-7.png")
if err != nil {
    fmt.Println(err)
}
display.PNG(img)

In [ ]:
img, err :=GetGraph("../data/optImg/optdigit_3-4.png")
if err != nil {
    fmt.Println(err)
}
display.PNG(img)

In [ ]:
img, err :=GetGraph("../data/optImg/optdigit_4-6.png")
if err != nil {
    fmt.Println(err)
}
display.PNG(img)


In [ ]:
//The first eight bytes of a PNG file always contain the following (decimal) values 137 80 78 71 13 10 26 10
// after 13 73 72 68 82
// after 0 0 0 8 - 0 0 0 8 -> dims
// 8 0 0 0 0 (bit dept = 8, color type - 0 = gray)
// idat 73 68 65 84
// hex 78 9C  or decimal 120 156 to start data --- 
// end 73 69 78 68
fmt.Println([]byte(img))

In [ ]:
func loadPng(imageName string) (image.Image, error) {
    imgfile, err := os.Open(imageName)

    if err != nil {
        fmt.Println("file not found",imageName)
    }
    defer imgfile.Close()
    return png.Decode(imgfile)
    
}

In [ ]:
// transform the png image in a vector where each position represents an intensity in a pixel
func vectorizeImage(img image.Image, number int)[65]int{
    var ret [65]int
    for i:=0;i<8;i++{
        for j:=0;j<8;j++{
            r,g,b,_:=img.At(j,i).RGBA()
            v := (int)(0.299*float64(r) + 0.587*float64(g) + 0.114*float64(b))/4095 // 256 of color palet and 16 of contrast augmentation
            ret[i*8+j] = v
        }
    }
    ret[64] = number
    return ret
}

In [ ]:
// we are going to create the dataset. each image is a line in my data set of 8x8=64 
// columns (we are going to interpret each column as a feature)
files, err := ioutil.ReadDir("../data/optImg/")
if err != nil {
    fmt.Println(err)
}
var matrix [][65]int
for _, f := range files {
    if strings.HasSuffix(f.Name(), ".png"){
        img,err:=loadPng("../data/optImg/"+f.Name())
        arr := strings.Split(strings.Split(f.Name(),"-")[1],".")
        data := vectorizeImage(img,strconv.Atoi(arr[0]))
        matrix = append(matrix,data)
    }
}

In [ ]:
fmt.Println(matrix[0])

Now we know we want 10 clusters. Use the k-means algorithm to find the clusters

In [ ]:
// Initialize a slice of gokmeans.Node's to
// hold our input data.
var data []gokmeans.Node
// Loop over the records creating our slice of
// gokmeans.Node's.
for _,record := range matrix {
    // Read in our record and check for errors.
    // Initialize a point.
    var point []float64
    // Fill in our point.
    for i := 0; i < 64; i++ {
        // Parse the float value.
        val:= float64(record[i])
      
        // Append this value to our point.
        point = append(point, val)
    }
    // Append our point to the data.
    data = append(data, point)
}

// Generate our clusters with k-means. 10 clusters, 1000 iterations
success, centroids := gokmeans.Train(data, 10, 1000)
if !success {
   fmt.Println("Could not generate clusters")
}


In [ ]:
// Output the centroids to stdout.
fmt.Println("The centroids for our clusters are:")
for _, centroid := range centroids {
    fmt.Println(centroid)
}

In [ ]:
// now we are going to build a confusion matrix.
var confusion [10][10]int
for index,record := range matrix {
    // Read in our record and check for errors.
    // Initialize a point.
    var point []float64
    // Fill in our point.
    for i := 0; i < 64; i++ {
        // Parse the float value.
        val:= float64(record[i])
        // Append this value to our point.
         point = append(point, val)
    }
    clusterIndex := gokmeans.Nearest(point, centroids) 
    expectedIndex := record[64]
    confusion[clusterIndex][expectedIndex]++
}



In [ ]:
// print values 
fmt.Println("\t Expected ")
fmt.Printf("\t0\t1\t2\t3\t4\t5\t6\t7\t8\t9 \n")
for i := 0 ;i<10; i++{
    fmt.Printf("%d\t%d\t%d\t%d\t%d\t%d\t%d\t%d\t%d\t%d\t%d\n",i,confusion[i][0],confusion[i][1],confusion[i][2],confusion[i][3],confusion[i][4],confusion[i][5],confusion[i][6],confusion[i][7],confusion[i][8],confusion[i][9])
}
